In [ ]:
!pip install corner
!pip install -U emcee

In [ ]:
import pandas as pd

import emcee
import corner

import matplotlib.gridspec as gridspec
import pickle

In [ ]:
GRB = pd.read_csv(os.path.join(dataDir,'GRB','GRB_data_Wang_et_al_2011.csv'))

In [ ]:
GRB.head()

,GRB,z,P_bolo,P_bolo_err,S_bolo,S_bolo_err,F_beam,F_beam_err,T_lag,T_lag_err,V,V_err,E_peak,E_peak_err_min,E_peak_err_max,T_RT,T_RT_err
0,970228,0.70,7.300000e-06,4.300000e-07,NaN,NaN,NaN,NaN,NaN,NaN,0.016,0.010,115.0,38.0,38.0,NaN,NaN
1,970508,0.84,3.300000e-06,3.300000e-07,0.000008,8.100000e-07,0.07950,0.02040,0.49,0.02,0.018,0.004,389.0,40.0,40.0,0.65,0.07
2,970828,0.96,1.000000e-05,1.100000e-06,0.000123,1.200000e-05,0.00532,0.00144,NaN,NaN,0.052,0.005,298.0,30.0,30.0,0.36,0.14
3,971214,3.42,7.500000e-07,2.400000e-08,NaN,NaN,NaN,NaN,0.03,0.05,0.048,0.002,190.0,20.0,20.0,NaN,NaN
4,980703,0.97,1.200000e-06,3.600000e-08,0.000028,2.900000e-06,0.01840,0.00267,0.69,0.02,0.024,0.001,254.0,25.0,25.0,3.00,0.19


In [ ]:
def calculate_dL(mu):
    '''
    dL is the luminosity distance of GRB, which can be obtained from the reconstructed distance moduli of Pantheon
    '''
    
    return 10**((mu-25)/5) * 10**6 * 3.086e+18

def calculate_dL_err(dL, mu_err):

    return abs(dL) * abs(np.log(10) * mu_err / 5.)

def calculate_L(d_L, P_bolo):

    '''
    Assuming that GRBs radiate isotropically, the isotropic equivalent
    luminosity can be derived from bolometric peak flux P_bolo
    '''

    return 4*np.pi*((d_L)**2)*P_bolo

def calculate_L_err(dL_err, dL, P_bolo_err, P_bolo, L):

    return abs(L) * np.sqrt((2. * dL_err / dL)**2 + (P_bolo_err / P_bolo)**2)

def calculate_E_iso(dL, S_bolo, z):
    '''
    The isotropic equivalent energy E_iso can be obtained from the 
    bolometric fluence S_bolo by
    '''

    return 4*np.pi*(dL)**2*S_bolo / (1+z)

def calculate_E_iso_err(L, dL_err, dL, S_bolo_err, S_bolo, z):
    
    return abs(L) * np.sqrt((2. * dL_err / dL)**2 + (S_bolo_err / S_bolo)**2) / (1+z)

def calculate_E_gamma(E_iso, F_beam):
    '''
    If GRBs radiate in two symmetric beams, then we can define the 
    collimation-corrected energy E_gamma as
    '''

    return E_iso * F_beam

def calculate_E_gamma_err(E_iso, F_beam, E_gamma, E_iso_err, F_beam_err):

    return abs(E_gamma) * np.sqrt((E_iso_err / E_iso)**2 + (F_beam_err / F_beam)**2 )

In [ ]:
# prepare sequential data
def strided_app(a, L, S ):  # Window len = L, Stride len/stepsize = S
    nrows = ((a.size-L)//S)+1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a, shape=(nrows,L), strides=(S*n,n))


# hyperparameters
BATCH_SIZE = 10
EPOCHS = 10000
sequence_len = 4
n_features = 1 
# number of neurons in LSTM
num_units = 100

def model_uncertainity(dropout_p=0.2):

    inputs = layers.Input((sequence_len, n_features))
    x = inputs
    
    x = layers.LSTM(100, return_sequences=True) (x)
    #x = layers.BatchNormalization() (x)
    x = layers.Dropout(dropout_p) (x, training=True)

    x = layers.LSTM(100, return_sequences=True) (x)
    #x = layers.BatchNormalization() (x)
    x = layers.Dropout(dropout_p) (x, training=True)

    x = layers.Dense(1) (x)

    return Model(inputs=inputs, outputs=x)

model_uncertainity = model_uncertainity()

model_uncertainity.load_weights(os.path.join(runDir,'cp_union2_1.hdf5'))

In [ ]:
zGRB_reconstruct = np.expand_dims(strided_app(GRB['z'].to_numpy(), 4, 4), axis=-1)

mu_reconstruct_uncertainity = []
n = 1000
for i in range(n):
    mu_reconstruct_uncertainity.append(model_uncertainity.predict(zGRB_reconstruct).flatten())

mu_reconstruct_uncertainity = np.array(mu_reconstruct_uncertainity)

GRB['mu'] = np.mean(mu_reconstruct_uncertainity, axis=0)
GRB['mu_err'] = np.std(mu_reconstruct_uncertainity, axis=0)

In [ ]:
# get distance moduli for redshifts using reconstructed mu-z funciton
GRB['dL'] = calculate_dL(GRB['mu'].to_numpy()) 
GRB['dL_err'] = calculate_dL_err(GRB['dL'].to_numpy(), GRB['mu_err'].to_numpy())
GRB['L'] = calculate_L(GRB['dL'].to_numpy(), GRB['P_bolo'].to_numpy()) 
GRB['L_err'] = calculate_L_err(GRB['dL_err'].to_numpy(), GRB['dL'].to_numpy(), GRB['P_bolo_err'].to_numpy(), GRB['P_bolo'].to_numpy(), GRB['L'].to_numpy()) 
GRB['E_iso'] = calculate_E_iso(GRB['dL'].to_numpy(), GRB['S_bolo'].to_numpy(), GRB['z'].to_numpy())
GRB['E_iso_err'] = calculate_E_iso_err(GRB['L'].to_numpy(), GRB['dL_err'].to_numpy(), GRB['dL'].to_numpy(), GRB['S_bolo_err'].to_numpy(), GRB['S_bolo'].to_numpy(), GRB['z'].to_numpy())
GRB['E_gamma'] = calculate_E_gamma(GRB['E_iso'].to_numpy(), GRB['F_beam'].to_numpy())
GRB['E_gamma_err'] = calculate_E_gamma_err(GRB['E_iso'].to_numpy(), GRB['F_beam'].to_numpy(), GRB['E_gamma'].to_numpy(), GRB['E_iso_err'].to_numpy(), GRB['F_beam_err'].to_numpy())

In [ ]:
# Since E_peak, error bars are asymmetric, just take the average of the lower and upper errors and then do the fit.
GRB['E_peak_err'] = GRB[['E_peak_err_min','E_peak_err_max']].mean(axis=1)

 quantities with a subscript ‘i’ indicate that they are measured in the comoving frame, which are related to the
quantities in the observer frame by τlag,i = τlag(1 + z)
−1
, τRT,i = τRT(1 + z)
−1
, Vi = V (1 + z) and Ep,i = Ep(1 + z)

In [ ]:
GRB['T_lag_i'] = GRB['T_lag'] / (1. + GRB['z'])
GRB['T_lag_i_err'] = GRB['T_lag_err'] / (1. + GRB['z'])
GRB['T_RT_i'] = GRB['T_RT'] / (1. + GRB['z'])
GRB['T_RT_i_err'] = GRB['T_RT_err'] / (1. + GRB['z'])
GRB['V_i'] = GRB['V'] * (1. + GRB['z'])
GRB['V_i_err'] = GRB['V_err'] * (1. + GRB['z'])
GRB['E_peak_i'] = GRB['E_peak'] * (1. + GRB['z'])
GRB['E_peak_i_err'] = GRB['E_peak_err'] * (1. + GRB['z'])
GRB['E_peak_i_err_min'] = GRB['E_peak_err_min'] * (1. + GRB['z'])
GRB['E_peak_i_err_max'] = GRB['E_peak_err_max'] * (1. + GRB['z'])

In [ ]:
# normalized
GRB['_T_lag_i'] = GRB['T_lag_i'] / 0.1
GRB['_T_lag_i_err'] = GRB['T_lag_i_err'] / 0.1
GRB['_V_i'] = GRB['V_i'] / 0.02
GRB['_V_i_err'] = GRB['V_i_err'] / 0.02
GRB['_E_peak_i'] = GRB['E_peak_i'] / 300
GRB['_E_peak_i_err'] = GRB['E_peak_i_err'] / 300
GRB['_E_peak_i_err_min'] = GRB['E_peak_i_err_min'] / 300
GRB['_E_peak_i_err_max'] = GRB['E_peak_i_err_max'] / 300
GRB['_T_RT_i'] = GRB['T_RT_i'] / 0.1
GRB['_T_RT_i_err'] = GRB['T_RT_i_err'] / 0.1

In [ ]:
# log transformation
GRB['log_T_lag_i'] = np.log10(GRB['_T_lag_i'])
GRB['log_T_lag_i_err'] = abs(GRB['_T_lag_i_err'] / (GRB['_T_lag_i'] * np.log(10)))
GRB['log_V_i'] = np.log10(GRB['_V_i'])
GRB['log_V_i_err'] = abs(GRB['_V_i_err'] / (GRB['_V_i'] * np.log(10)))
GRB['log_E_peak_i'] = np.log10(GRB['_E_peak_i'])
GRB['log_E_peak_i_err'] = abs(GRB['_E_peak_i_err'] / (GRB['_E_peak_i'] * np.log(10)))
GRB['log_E_peak_i_err_min'] = abs(GRB['_E_peak_i_err_min'] / (GRB['_E_peak_i'] * np.log(10)))
GRB['log_E_peak_i_err_max'] = abs(GRB['_E_peak_i_err_max'] / (GRB['_E_peak_i'] * np.log(10)))
GRB['log_T_RT_i'] = np.log10(GRB['_T_RT_i'])
GRB['log_T_RT_i_err'] = abs(GRB['_T_RT_i_err'] / (GRB['_T_RT_i'] * np.log(10)))

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
GRB['log_L'] = np.log10(GRB['L'])
GRB['log_L_err'] = abs(GRB['L_err'] / (GRB['L'] * np.log(10)))
GRB['log_E_iso'] = np.log10(GRB['E_iso'])
GRB['log_E_iso_err'] = abs(GRB['E_iso_err'] / (GRB['E_iso'] * np.log(10)))
GRB['log_E_gamma'] = np.log10(GRB['E_gamma'])
GRB['log_E_gamma_err'] = abs(GRB['E_gamma_err'] / (GRB['E_gamma'] * np.log(10)))

In [ ]:
# split the data in to low-z, high-z and All-z
low_z_GRB = GRB[GRB['z']<=1.4]
high_z_GRB = GRB[GRB['z']>1.4]
All_z_GRB = GRB

In [ ]:
# filter GRBs seperately for each correlation, such that required data are present for each GRB
 

In [ ]:
def log_likelihood(theta, x, y, xerr, yerr):
    b, a, sigma_int = theta
    model = b * x + a
    sigma2 = sigma_int**2 + yerr**2 + b**2 * xerr**2
    return -0.5 * np.sum((y - model) ** 2 / sigma2 + np.log(sigma2))

def log_prior(theta):
    b, a, sigma_int = theta
    if sigma_int > 0:
        return 0.0
    return -np.inf

def log_posterior(theta, x, y, xerr, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, xerr, yerr)

In [ ]:
correlations = {
    'T_lag-L' : ('log_T_lag_i', 'log_L', 'log_T_lag_i_err', 'log_L_err'),
    'V-L' : ('log_V_i', 'log_E_iso', 'log_V_i_err', 'log_E_iso_err'),
    'E_peak-L' : ('log_E_peak_i', 'log_L', 'log_E_peak_i_err', 'log_L_err'),
    'E_peak-E_gamma' : ('log_E_peak_i', 'log_E_gamma', 'log_E_peak_i_err', 'log_E_gamma_err'),
    'T_RT-L' : ('log_T_RT_i', 'log_L', 'log_T_RT_i_err', 'log_L_err'),
    'E_peak-E_iso' : ('log_E_peak_i', 'log_E_iso', 'log_E_peak_i_err', 'log_E_iso_err')
}

# Calculate best fit line with uncertainities(Bayesian approach) using emcee
GRB_samples = (low_z_GRB, high_z_GRB, All_z_GRB)
sample_types =  ('low-z', 'high-z', 'All-z')
colors = ('b','r','k')

# MCMC parameters
nwalkers, ndim = 64, 3
nsteps, nburns = 10000, 5000

# empty dictionary to save best fit parameters and uncertainities
BestFitParameters = { 
    'T_lag-L':{'low-z':None, 'high-z':None, 'All-z':None},
    'V-L':{'low-z':None, 'high-z':None, 'All-z':None},
    'E_peak-L':{'low-z':None, 'high-z':None, 'All-z':None},
    'E_peak-E_gamma':{'low-z':None, 'high-z':None, 'All-z':None},
    'T_RT-L':{'low-z':None, 'high-z':None, 'All-z':None},
    'E_peak-E_iso':{'low-z':None, 'high-z':None, 'All-z':None}
}

for i, correlation in enumerate(correlations):

    luminosities = correlations[correlation]

    '''
    # create empty figure object for time series (steps) plot of parameters in chain
    fig1 = plt.figure(constrained_layout=True, figsize=(15,5))
    gs1 = gridspec.GridSpec(nrows=1, ncols=3, figure=fig1)
    '''

    # create empty figure object for corner plots (confidence contours and marginalized PDFs of parameters)
    fig2 = plt.figure(figsize=(5, 5))
    fig2.patch.set_facecolor('white')

    for k, (GRB_sample, sample_type, color) in enumerate(zip(GRB_samples, sample_types, colors)):

        df = GRB_sample.filter(luminosities).dropna()
        
        x = df[luminosities[0]].to_numpy()
        y = df[luminosities[1]].to_numpy()
        xerr = df[luminosities[2]].to_numpy()
        yerr = df[luminosities[3]].to_numpy()

        starting_guesses = np.random.rand(nwalkers, ndim)
        
        sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(x, y, xerr, yerr))
        sampler.run_mcmc(starting_guesses, nsteps)

        labels = ['b', 'a', 'sigma_int']
        '''
        # time series plot of parameters
        samples = sampler.get_chain()
        gs11 = gridspec.GridSpecFromSubplotSpec(nrows=3, ncols=1, subplot_spec=gs1[k])

        for j in range(ndim):
            ax = fig1.add_subplot(gs11[j])
            ax.plot(samples[..., j], 'k', alpha=0.3)
            ax.set_xlim(0, len(samples))
            ax.set_ylabel(labels[j])
            ax.yaxis.set_label_coords(-0.1, 0.5)
        #axes[-1].set_xlabel("step number");
        '''
        # corner plots
        flat_samples = sampler.get_chain(discard=nburns, flat=True)
        corner.corner(flat_samples, labels=labels, color=color,fig=fig2)

        # save best fit values(mean) and uncertainities(std) of parameters in a dictionary
        BestFitParameters[correlation][sample_type] = {
                'a' : np.mean(flat_samples[:, 1]), 
                'a_err' : np.std(flat_samples[:, 1]),
                'b' : np.mean(flat_samples[:, 0]), 
                'b_err' : np.std(flat_samples[:, 0]),
                'sigma_int' : np.mean(flat_samples[:, 2]),
                'sigma_int_err' : np.std(flat_samples[:, 2])
        }
    
        fig2.axes[0].annotate(sample_type, xy=(0.95*2.5, 0.95-k*0.2), xycoords='axes fraction',color=color)
  
    fig2.suptitle(correlation)

    #fig1.savefig('time_series_of_params.png')
    fig2.savefig(correlation+'_corner_plot.png')
    #fig1.show()
    fig2.show()

In [ ]:
with open(os.path.join(runDir, 'BestFitParameters.pickle'), 'wb') as handle:
    pickle.dump(BestFitParameters, handle)

In [ ]:
with open(os.path.join(runDir, 'BestFitParameters.pickle'), 'rb') as handle:
    BestFitParameters = pickle.load(handle)

In [ ]:
def flatten_dict(nested_dict):
    res = {}
    if isinstance(nested_dict, dict):
        for k in nested_dict:
            flattened_dict = flatten_dict(nested_dict[k])
            for key, val in flattened_dict.items():
                key = list(key)
                key.insert(0, k)
                res[tuple(key)] = val
    else:
        res[()] = nested_dict
    return res


def nested_dict_to_df(values_dict):
    flat_dict = flatten_dict(values_dict)
    df = pd.DataFrame.from_dict(flat_dict, orient="index")
    df.index = pd.MultiIndex.from_tuples(df.index)
    df = df.unstack(level=-1)
    df.columns = df.columns.map("{0[1]}".format)
    return df

table = nested_dict_to_df(BestFitParameters)

!pip install tabulate

from tabulate import tabulate

print(tabulate(table, headers='keys', tablefmt='fancy_grid'))

In [ ]:
correlations = {
    'T_lag-L' : {'features': ('log_T_lag_i', 'log_L', 'log_T_lag_i_err', 'log_L_err'), 'ylim':(50, 54), 'xlim':(-2.0,2.0)},
    'V-L' : {'features': ('log_V_i', 'log_L', 'log_V_i_err', 'log_L_err'), 'ylim':(50, 54), 'xlim':(-1.0,2.0)},
    'E_peak-L' : {'features': ('log_E_peak_i', 'log_L', 'log_E_peak_i_err', 'log_L_err'), 'ylim':(49, 54), 'xlim':(-1.5,1.5)},
    'E_peak-E_gamma' : {'features': ('log_E_peak_i', 'log_E_gamma', 'log_E_peak_i_err', 'log_E_gamma_err'), 'ylim':(48, 52), 'xlim':(-1.5,1.5)},
    'T_RT-L' : {'features': ('log_T_RT_i', 'log_L', 'log_T_RT_i_err', 'log_L_err'), 'ylim':(50, 54), 'xlim':(-1.0,2.0)},
    'E_peak-E_iso' : {'features': ('log_E_peak_i', 'log_E_iso', 'log_E_peak_i_err', 'log_E_iso_err'), 'ylim':(50, 55), 'xlim':(-1.5,1.5)}
}

GRB_samples = (low_z_GRB, high_z_GRB, All_z_GRB)
sample_types =  ('low-z', 'high-z', 'All-z')
colors = ('b','r','k')

fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(15,20))
fig.patch.set_facecolor('white')
# plot best fit for each luminosity correlation
for i, correlation in enumerate(correlations):

    luminosities = correlations[correlation]['features']

    # create empty figures with no axis
    #plt.figure(figsize=(15,5))

    #plt.subplot(121)

    for GRB_sample, sample_type, color in zip(GRB_samples, sample_types, colors):
        
        df = GRB_sample.filter(luminosities).dropna()
            
        x = df[luminosities[0]].to_numpy()
        y = df[luminosities[1]].to_numpy()
        xerr = df[luminosities[2]].to_numpy()
        yerr = df[luminosities[3]].to_numpy()

        if sample_type != 'All-z':
            axs[int(i/2), i%2].errorbar(x,y,xerr=xerr,yerr=yerr,fmt='.', ecolor=color, label=sample_type)
        
        axs[int(i/2), i%2].plot(
            np.linspace(-2,2,100), 
            BestFitParameters[correlation][sample_type]['a'] + BestFitParameters[correlation][sample_type]['b'] * np.linspace(-2,2,100),
            #luminosity_correlation_fit(np.linspace(-2,2,100), 'T_lag-L', sample_type), 
            linestyle='-', color=color, label=sample_type +' best fit'
            )

    axs[int(i/2), i%2].set_title(correlation)
    axs[int(i/2), i%2].set_xlabel(luminosities[0])
    axs[int(i/2), i%2].set_ylabel(luminosities[1])
    axs[int(i/2), i%2].set_ylim(correlations[correlation]['ylim'])
    axs[int(i/2), i%2].set_xlim(correlations[correlation]['xlim'])
        
    axs[int(i/2), i%2].legend()

fig.show()

In [ ]:
# E_peak -> E_gamma -> E_iso -> d_L

def caliberate_d_L(log_E_peak_i, log_E_peak_i_err, F_beam, F_beam_err, S_bolo, S_bolo_err, z):

    def calculate_E_gamma_from_correlation(log_E_peak_i, log_E_peak_i_err):

        # get best fit parameters for correlation
        a = BestFitParameters['E_peak-E_gamma']['All-z']['a']
        a_err = BestFitParameters['E_peak-E_gamma']['All-z']['a_err']
        b = BestFitParameters['E_peak-E_gamma']['All-z']['b']
        b_err = BestFitParameters['E_peak-E_gamma']['All-z']['b_err']

        # log transformed, normalized E_peak
        log_E_gamma = a + b * log_E_peak_i
        log_E_gamma_err = np.sqrt(a_err**2 + (abs(b * log_E_peak_i) * np.sqrt((b_err/b)**2 + (log_E_peak_i_err/log_E_peak_i)**2))**2)

        E_gamma = 10 ** log_E_gamma

        E_gamma_err = abs(E_gamma) * abs(np.log(10) * log_E_gamma_err)

        return E_gamma, E_gamma_err

    def calculate_E_iso(E_gamma, E_gamma_err, F_beam, F_beam_err):

        E_iso = E_gamma / F_beam
        E_iso_err = abs(E_iso) * np.sqrt( (E_gamma_err/E_gamma)**2 + (F_beam_err/F_beam)**2 )

        return E_iso, E_iso_err
    

    def calculate_d_L(E_iso, E_iso_err, S_bolo, S_bolo_err, z):

        # calculate d_L
        a = E_iso * (1. + z)
        a_err = (1.+z) * E_iso_err

        b = 4. * np.pi * S_bolo
        b_err = 4. * np.pi * S_bolo_err

        c = a/b
        c_err = abs(c) * np.sqrt((a_err/a)**2 + (b_err/b)**2)

        d_L = np.sqrt(c) 
        d_L_err = c_err / (2*np.sqrt(c))
    
        return d_L, d_L_err
    
    # calculate E_gamma from E_gamma-E_peak correlation
    E_gamma, E_gamma_err = calculate_E_gamma_from_correlation(log_E_peak_i, log_E_peak_i_err)

    # calculate E_iso
    E_iso, E_iso_err = calculate_E_iso(E_gamma, E_gamma_err, F_beam, F_beam_err)

    # calculate d_L
    d_L, d_L_err = calculate_d_L(E_iso, E_iso_err, S_bolo, S_bolo_err, z)

    return d_L, d_L_err

def caliberate_mu(d_L, d_L_err):

    # calculate mu
    mu = 5. * np.log10(d_L / (10**6 * 3.086e+18)) + 25.
    mu_err = abs(5. * d_L_err / (d_L * np.log(10))) 

    return mu, mu_err

In [ ]:
# caliberate d_L from E_peak - E_gamma relation, hence only those GRBs with 
# sufficient data(E_peak, F_beam, S_bolo) can be caliberated using from this 
filtered_GRB = GRB[['E_gamma', 'E_gamma_err','log_E_peak_i', 'log_E_peak_i_err', 'F_beam', 'F_beam_err', 'S_bolo', 'S_bolo_err', 'z']].dropna()

d_L, d_L_err = caliberate_d_L(
    filtered_GRB['log_E_peak_i'].to_numpy(), 
    filtered_GRB['log_E_peak_i_err'].to_numpy(),
    filtered_GRB['F_beam'].to_numpy(),
    filtered_GRB['F_beam_err'].to_numpy(),
    filtered_GRB['S_bolo'].to_numpy(),
    filtered_GRB['S_bolo_err'].to_numpy(),
    filtered_GRB['z'].to_numpy())

mu_GRB, mu_err_GRB = caliberate_mu(d_L, d_L_err)

fig, ax = plt.subplots(figsize=(10, 8))
fig.patch.set_facecolor('white')

ax.errorbar(filtered_GRB['z'].to_numpy(), mu_GRB, yerr=mu_err_GRB, fmt='.', color='b', capsize=2, label='GRB')
ax.errorbar(data1['zCMB'], data1['MU'], yerr=data1['MUERR'], fmt='.r', capsize=2, label='Pantheon sample')
ax.legend()
ax.set_xlabel('z')
ax.set_ylabel('mu')
plt.show()

In [ ]:
# estimate omega_M and omega_L

# calculate H(z) for all z

# speed of light
c = 9.7 * 10e-15 # Mpc/sec
z = filtered_GRB['z'].to_numpy()
H_z = c * (1. + z)**2 * np.log(1.+z) / d_L
H_z_err = abs(H_z) * d_L_err / d_L
H_0 = 67.4 # km/sec/Mpc

# 

In [ ]:
def log_likelihood(theta, x, y, xerr, yerr):
    b, a, sigma_int = theta
    model = b * x + a
    sigma2 = sigma_int**2 + yerr**2 + b**2 * xerr**2
    return -0.5 * np.sum((y - model) ** 2 / sigma2 + np.log(sigma2))

def log_prior(theta):
    b, a, sigma_int = theta
    if sigma_int > 0:
        return 0.0
    return -np.inf

def log_posterior(theta, x, y, xerr, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, xerr, yerr)

# MCMC parameters
nwalkers, ndim = 64, 3
nsteps, nburns = 10000, 5000


# create empty figure object for corner plots (confidence contours and marginalized PDFs of parameters)
fig2 = plt.figure(figsize=(5, 5))
fig2.patch.set_facecolor('white')

x = (1. + z)**3
y = (1. + z)**2 #(H_z / H_0)**2

xerr = 0
yerr = 0 #abs(y * 2 * H_z_err / H_z)# should errors in H_0 be considered ?

starting_guesses = np.random.rand(nwalkers, ndim)
        
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(x, y, xerr, yerr))
sampler.run_mcmc(starting_guesses, nsteps)

labels = ['omega_L', 'omega_M','sigma_int']
'''
# time series plot of parameters
samples = sampler.get_chain()
gs11 = gridspec.GridSpecFromSubplotSpec(nrows=3, ncols=1, subplot_spec=gs1[k])

for j in range(ndim):
    ax = fig1.add_subplot(gs11[j])
    ax.plot(samples[..., j], 'k', alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[j])
    ax.yaxis.set_label_coords(-0.1, 0.5)
    #axes[-1].set_xlabel("step number");
'''
# corner plots
flat_samples = sampler.get_chain(discard=nburns, flat=True)
corner.corner(flat_samples[:,:2], labels=labels[:2], color=color,fig=fig2)

# save best fit values(mean) and uncertainities(std) of parameters in a dictionary
omega_M_omega_L_correlation_best_fit = {
        'omega_L' : np.mean(flat_samples[:, 1]), 
        'omega_L_err' : np.std(flat_samples[:, 1]),
        'omega_M' : np.mean(flat_samples[:, 0]), 
        'omega_M_err' : np.std(flat_samples[:, 0]),
        #'sigma_int' : np.mean(flat_samples[:, 2]),
        #'sigma_int_err' : np.std(flat_samples[:, 2])
    }

#fig2.axes[0].annotate(sample_type, xy=(0.95*2.5, 0.95-k*0.2), xycoords='axes fraction',color=color)

fig2.suptitle('omega_M- omega_L')

#fig1.savefig('time_series_of_params.png')
fig2.savefig('omega_M_omega_L_corner_plot.png')
#fig1.show()
fig2.show()

In [ ]:
def log_likelihood(theta, x, y):
    b = theta
    model = b * x + 1
    return -0.5 * np.sum((y - model) ** 2)

def log_prior(theta):
    b = theta
    return 0.

def log_posterior(theta, x, y):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y)

# MCMC parameters
nwalkers, ndim = 64, 1
nsteps, nburns = 10000, 5000


# create empty figure object for corner plots (confidence contours and marginalized PDFs of parameters)
fig2 = plt.figure(figsize=(5, 5))
fig2.patch.set_facecolor('white')

x = (1. + z)**3 - 1.
y = (1. + z)**2 #(H_z / H_0)**2

xerr = 0
yerr = 0 #abs(y * 2 * H_z_err / H_z)# should errors in H_0 be considered ?

starting_guesses = np.random.rand(nwalkers, ndim)
        
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(x, y))
sampler.run_mcmc(starting_guesses, nsteps)

labels = ['omega_L', 'omega_M','sigma_int']
'''
# time series plot of parameters
samples = sampler.get_chain()
gs11 = gridspec.GridSpecFromSubplotSpec(nrows=3, ncols=1, subplot_spec=gs1[k])

for j in range(ndim):
    ax = fig1.add_subplot(gs11[j])
    ax.plot(samples[..., j], 'k', alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[j])
    ax.yaxis.set_label_coords(-0.1, 0.5)
    #axes[-1].set_xlabel("step number");
'''
# corner plots
flat_samples = sampler.get_chain(discard=nburns, flat=True)
corner.corner(flat_samples, labels='omega_M', color=color,fig=fig2)

# save best fit values(mean) and uncertainities(std) of parameters in a dictionary
omega_M_omega_L_correlation_best_fit = {
        'omega_L' : np.mean(flat_samples), 
        'omega_L_err' : np.std(flat_samples),
        'omega_M' : np.mean(flat_samples), 
        'omega_M_err' : np.std(flat_samples),
        #'sigma_int' : np.mean(flat_samples[:, 2]),
        #'sigma_int_err' : np.std(flat_samples[:, 2])
    }

#fig2.axes[0].annotate(sample_type, xy=(0.95*2.5, 0.95-k*0.2), xycoords='axes fraction',color=color)

fig2.suptitle('omega_M')

#fig1.savefig('time_series_of_params.png')
fig2.savefig('omega_M_corner_plot.png')
#fig1.show()
fig2.show()

In [ ]:
print('omega_M = {}'.format(omega_M_omega_L_correlation_best_fit['omega_M']))
print('omega_M_err = {}'.format(omega_M_omega_L_correlation_best_fit['omega_M_err']))

#print('omega_L = {}'.format(omega_M_omega_L_correlation_best_fit['omega_L']))
#print('omega_L_err = {}'.format(omega_M_omega_L_correlation_best_fit['omega_L_err']))